In [1]:
import os
import torch
import torchvision
from torchsummary import summary
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader as dataloader
from tensorboardX import SummaryWriter
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import numpy as np
import time
import csv

# 设定参数

In [2]:
# hyper parameters
path_model = "./checkpoint/"
batch_size = 512
epochs = 15
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
# sampler = torch.utils.data.SubsetRandomSampler(indices=list(range(2000)))

# download mnist dataset
dataset_train = torchvision.datasets.MNIST(root='./data/',train=True,download=True,transform=transform)
dataset_test = torchvision.datasets.MNIST(root='./data/',train=False,download=True,transform=transform)

class_names = dataset_train.classes  # 获取数据集的分类信息 返回一个字典
# load dataset
data_train = dataloader(dataset=dataset_train,batch_size=batch_size,shuffle=False)
data_test = dataloader(dataset=dataset_test,batch_size=batch_size,shuffle=True)

print(len(data_train))
# for batch_idx,(data,target) in enumerate(data_test):
#     print("id ",batch_idx, "data shape",data.shape)


118


# 定义训练过程

In [53]:
def test():
    net.eval()  # 切换到测试模式
    test_correct_num = 0
    with torch.no_grad():   # 不更新参数

        for batch_idx,(data,target) in enumerate(data_test):
            # data = data.to(device)
            # target = target.to(device)
            output, _ = net(data) # 正向传播得到预测值
            _, pred = torch.max(output.data, 1)
            test_correct_num += torch.sum(pred==target).item()
            print("Test Epoch:{} [{}/{} ({:.0f}%)]\t acc:{:.2f}".format(epoch,batch_idx*batch_size,len(data_test.dataset),
                                                 100. * batch_size*batch_idx/len(data_test.dataset),test_correct_num/len(data_test.dataset)))
def train(epoch):

    for batch_idx, (data, target) in enumerate(data_train):
        # 清除grad累积值
        optimizer.zero_grad()
        # 读取dataloader中的数据，前半部分是tensor变量，后半部分是真实label
        data = data.to(device)
        target = target.to(device)
        # forward之后得到预测值
        output, process_output_ = net(data)
        if (epoch == epochs - 1) & (batch_idx == len(data_train)-1):
            save_layerout(epoch, batch_idx, process_output_)
        # 计算loss
        loss = cost_fun(output, target)
        # backward
        loss.backward()
        # 收集一组新的梯度，并使用optimizer.step()将其传播回每个网络参数
        optimizer.step()
        # 给出loss和acc
        train_loss.append(loss)
        _, pred = torch.max(output.data, 1)
        correct_num = torch.sum(pred == target).item()
        train_acc.append(correct_num / batch_size)
        print("Train Epoch:{}[{}/{} ({:.0f}%)]\t Loss:{:.6f} acc:{:.2f}".format(epoch, batch_idx * batch_size,
               len(data_train.dataset),100. * batch_size * batch_idx / len(data_train.dataset), loss.item(),correct_num / batch_size))
        
        # MNIST一共60000个数据，batch_size为512，一共60000/512 = 118个batch
        if (epoch == epochs - 1) & (batch_idx == len(data_train)-1):
            save_weight(epoch, batch_idx)

def save_layerout(epoch, batch_idx, process_output_):
    cscfphn_str = str(epoch) + '_' + str(batch_idx) + '_'  # curt_step_curt_file_pre_head_name
    # 将网络的过程结果进行保存
    col_counter = 0
    for key in process_output_.keys():

        # 获取变量的尺寸，根据其尺寸定义前导尺寸值
        curt_value_shape = process_output_[key].shape
        curt_value_shape_len = len(curt_value_shape)
        pre_load_len_name = ''
        # --
        for cvsl_idx in range(curt_value_shape_len):
            pre_load_len_name = pre_load_len_name + str(curt_value_shape[cvsl_idx]) + '_'
        # --
        dif_curt_value_shape_len = 4 - curt_value_shape_len
        if dif_curt_value_shape_len:
            for dcvsl_idx in range(dif_curt_value_shape_len):
                pre_load_len_name = '0_' + pre_load_len_name

        # 获取文件名
        curt_csv_file_name = cscfphn_str + str(col_counter) + '_' + pre_load_len_name + key
        # 数据存储
        if not os.path.isdir('layeroutput_data'):
            os.mkdir('.\layeroutput_data')
        layer_output_result_path = '.\layeroutput_data'
        with open(layer_output_result_path + '\\' + curt_csv_file_name + ".csv", "w+",
                    newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=' ')
            data = process_output_[key].reshape(-1)
            data_np = data.detach().numpy()  # TODO 注意确认是否存在通道问题、此处摒弃梯度信息
            writer.writerows([data_np])
            # writer.writerows([data_curt] for data_curt in data_np)
            csvfile.close()
            # --
        col_counter += 1

def save_weight(epoch, batch_idx):
#    for step, (batch_x, batch_y) in enumerate(loader):
    cscfphn_str = str(epoch) + '_' + str(batch_idx) + '_'  # curt_step_curt_file_pre_head_name

    # 网络参数输出
    parm = {}
    for name, parameters in net.named_parameters():
        # print('网络参数输出')
        # print(name, ':', parameters.size())
        parm[name] = parameters.detach().numpy()  # 将tensor变量转换为np格式

        # TODO 将读取出来的参数进行保存，注意确认是否存在tensor与np之间的转换通道问题
        '''
        文件名共分8部分：
        epoch step index                   number        channel   length  width  value_name
        轮数   步数  序号(按参数在网络中的顺序)  变量中数据块个数 变量通道数  变量长   变量宽  变量名称
        '''
    col_counter = 0
    for key in parm.keys():
        # 获取变量的尺寸，根据其尺寸定义前导尺寸值
        curt_value_shape = parm[key].shape
        curt_value_shape_len = len(curt_value_shape)
        pre_load_len_name = ''
        # -- 定义有信息的前导尺寸
        for cvsl_idx in range(curt_value_shape_len):
            pre_load_len_name = pre_load_len_name + str(curt_value_shape[cvsl_idx]) + '_'
        # -- 将无信息的前导尺寸设置为缺省值
        dif_curt_value_shape_len = 4 - curt_value_shape_len
        if dif_curt_value_shape_len:
            for dcvsl_idx in range(dif_curt_value_shape_len):
                pre_load_len_name = '0_' + pre_load_len_name

        # 获取文件名
        curt_csv_file_name = cscfphn_str + str(col_counter) + '_' + pre_load_len_name + key

        # 数据存储
        # 创建文件对象
        if not os.path.isdir('layer_para'):
            os.mkdir('.\layer_para')
        if not os.path.isdir('layer_para\\bias'):
            os.mkdir('.\layer_para\\bias')
        if not os.path.isdir('layer_para\\weight'):
            os.mkdir('.\layer_para\\weight')
        layer_para_path = '.\layer_para'
        layer_bias_para_path = '.\layer_para\\bias'
        layer_weight_para_path = '.\layer_para\\weight'
        if('bias' in key):
            with open(layer_bias_para_path + '\\' + curt_csv_file_name + ".csv", "w+", newline='') as csvfile:
            # 基于文件对象构建 csv写入对象
                writer = csv.writer(csvfile, delimiter=' ')
                data = parm[key].reshape(-1)
                # writer.writerows([data_curt] for data_curt in data)
                writer.writerows([data])
                csvfile.close()
        else :
            with open(layer_weight_para_path + '\\' + curt_csv_file_name + ".csv", "w+", newline='') as csvfile:
            # 基于文件对象构建 csv写入对象
                writer = csv.writer(csvfile, delimiter=' ')
                data = parm[key].reshape(-1)
                # writer.writerows([data_curt] for data_curt in data)
                writer.writerows([data])
                csvfile.close()            
        # --
        col_counter += 1 
    

def save_state():
    print('===> Saving weights...')
    state = {
        'state': net.state_dict(),
        'epoch': epoch  # 将epoch一并保存
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('./checkpoint')
    torch.save(state, path_model + 'Epoch:' + str(epoch) + ' Loss:' + str(train_loss[-1].item()) + '.pth')

def predict():
    state_path = './checkpoint/model_14.pth' #  ***为指定加载的权重文件名称
    print('===> Loading weights : ' + state_path)
    weight_dict = torch.load(state_path)  # 加载最后训练出的权重
    net.load_state_dict(weight_dict['state'])
    # 从测试集中选取一个batch做预测
    # pred_test = enumerate(data_test)
    # batch_idx, (pred_data, pred_gt) = next(pred_test)
    # output = net(pred_data)
    # print("data: ", output.data)
    # maxdata, pred = torch.max(output.data, 1) # 得到预测值,返回每一行的最大值，且返回索引
    # print("maxdata: ", maxdata)
    # print("ground truth: ",pred_gt)
    # print("predict value: ",pred)
    # 获取预测结果
    classes = [
        "0",
        "1",
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8",
        "9",
    ]


    # 把tensor转成Image， 方便可视化
    show = ToPILImage()

    device = "cuda" if torch.cuda.is_available() else 'cpu'
    net.eval()
    for i in np.random.randint(0,20,size=10):
        x, y = dataset_test[i][0], dataset_test[i][1]
        # tensor格式数据可视化
        show(x).show()
        # 扩展张量维度为4维
        x = Variable(torch.unsqueeze(x, dim=0).float(), requires_grad=False).to(device)
        with torch.no_grad():
            pred = net(x)
            # 得到预测类别中最高的那一类，再把最高的这一类对应classes中的哪一类标签
            predicted, actual = classes[torch.argmax(pred[0])], classes[y]
            # 最终输出的预测值与真实值
            print(f'predicted: "{predicted}", actual:"{actual}"')


# 构建网络

In [7]:
# 构建网络
class LeNet(nn.Module): 					# 继承于nn.Module这个父类
    def __init__(self):						# 初始化网络结构
        super(LeNet, self).__init__()    	# 多继承需用到super函数
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),  # 输出为6*28*28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 输出为6*14*14
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),  # 输出为16*10*10
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 输出为16*5*5
        )
        self.block_2 = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
            nn.Softmax(dim=1),
        )

    def forward(self, x):  # 正向传播过程
        # x = self.block_1(x)
        # x = x.view(-1,16*5*5)
        # x = self.block_2(x)
        block1_conv1_out = self.block_1[0](x)
        block1_relu1_out = self.block_1[1](block1_conv1_out)
        block1_maxpo1_out = self.block_1[2](block1_relu1_out)
        block1_conv2_out = self.block_1[3](block1_maxpo1_out)
        block1_relu2_out = self.block_1[4](block1_conv2_out)
        block1_maxpo2_out = self.block_1[5](block1_relu2_out)

        block1_maxpo2_out = block1_maxpo2_out.view(-1,16*5*5)

        block2_fc1_out = self.block_2[0](block1_maxpo2_out)
        block2_relu1_out = self.block_2[1](block2_fc1_out)
        block2_fc2_out = self.block_2[2](block2_relu1_out)
        block2_relu2_out = self.block_2[3](block2_fc2_out)
        block2_fc3_out = self.block_2[4](block2_relu2_out)

        softmax_output =  self.block_2[5](block2_fc3_out)

        process_output = {'block1_conv1_out': block1_conv1_out,
                          'block1_relu1_out': block1_relu1_out,
                          'block1_maxpo1_out': block1_maxpo1_out,
                          'block1_conv2_out': block1_conv2_out,
                          'block1_relu2_out': block1_relu2_out,
                          'block1_maxpo2_out': block1_maxpo2_out,

                          'block2_fc1_out': block2_fc1_out,
                          'block2_relu1_out': block2_relu1_out,
                          'block2_fc2_out': block2_fc2_out,
                          'block2_relu2_out': block2_relu2_out,
                          'block2_fc3_out': block2_fc3_out,
                          'softmax_output': softmax_output
                        }

        return block2_fc3_out, process_output


# 主函数

In [54]:
if __name__ == '__main__':
    writer = SummaryWriter('logs')
    # device = torch.device('cuda:0')

    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    
    net = LeNet()  # 实例化网络
    # data_input = Variable(torch.randn(16,1,28,28))
    # print(net(data_input))
    net.to(device) # 将参数送入GPU中

    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            torch.nn.init.xavier_uniform_(m.weight)

    net.apply(init_weights)

    cost_fun = nn.CrossEntropyLoss()
    # optim
    optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.95, weight_decay=1e-3)

    # train
    for epoch in range(epochs):

        # train
        train_loss = []
        train_acc = []
        # train
        train(epoch)
        writer.add_scalar('Train/Loss', train_loss[-2].item(), epoch)
        writer.add_scalar('Train/Acc', train_acc[-2], epoch)

        # ----------------------------------------- #
        # save_state
        # ----------------------------------------- #
        print('===> Saving models...')
        state = {
            'state': net.state_dict(),
            'epoch': epoch  # 将epoch一并保存
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('./checkpoint')
        torch.save(state, path_model + 'Epoch-' + str(epoch) + '-Loss-'+ str(train_loss[-1].item()) + '.pth')

        # ----------------------------------------- #
        # test
        # ----------------------------------------- #
        test()
    writer.close()

    # ----------------------------------------- #
    # 加载指定的weights进行预测
    # ----------------------------------------- #
    # predict()

Train Epoch:0[0/60000 (0%)]	 Loss:2.299441 acc:0.10
Train Epoch:0[512/60000 (1%)]	 Loss:2.301670 acc:0.09
Train Epoch:0[1024/60000 (2%)]	 Loss:2.302093 acc:0.10
Train Epoch:0[1536/60000 (3%)]	 Loss:2.293128 acc:0.11
Train Epoch:0[2048/60000 (3%)]	 Loss:2.297524 acc:0.11
Train Epoch:0[2560/60000 (4%)]	 Loss:2.300138 acc:0.10
Train Epoch:0[3072/60000 (5%)]	 Loss:2.301613 acc:0.10
Train Epoch:0[3584/60000 (6%)]	 Loss:2.294523 acc:0.09
Train Epoch:0[4096/60000 (7%)]	 Loss:2.299752 acc:0.09
Train Epoch:0[4608/60000 (8%)]	 Loss:2.295251 acc:0.09
Train Epoch:0[5120/60000 (9%)]	 Loss:2.296507 acc:0.09
Train Epoch:0[5632/60000 (9%)]	 Loss:2.290614 acc:0.11
Train Epoch:0[6144/60000 (10%)]	 Loss:2.286134 acc:0.09
Train Epoch:0[6656/60000 (11%)]	 Loss:2.297364 acc:0.09
Train Epoch:0[7168/60000 (12%)]	 Loss:2.288844 acc:0.12
Train Epoch:0[7680/60000 (13%)]	 Loss:2.288592 acc:0.10
Train Epoch:0[8192/60000 (14%)]	 Loss:2.288130 acc:0.11
Train Epoch:0[8704/60000 (15%)]	 Loss:2.283421 acc:0.12
Train Ep

# 测试

In [4]:
import cv2
from torchvision.transforms import ToPILImage

In [86]:
def predictOneTensor(model_path, file_path, labely):
    device = "cuda" if torch.cuda.is_available() else 'cpu'

    img0 = cv2.imread(file_path, 0).astype(np.uint8)
    print(img0.shape)
    img0 = cv2.resize(img0, (28,28))
    img0 = img0 / 255.
    img0 = torch.from_numpy(img0)

    img = torch.as_tensor(img0, dtype=torch.float32)
    img = img.unsqueeze(0) #在第一维度上增加一个维度，作为通道大小
    img = Variable(torch.unsqueeze(img, dim=0).float(),).to(device) #在第一维度上增加一个维度，作为batch size大小
    # img = img.permute(0, 3, 1, 2) # 将图像channel提到前面即 [batch size, width, height, channel]-> [batch size, channel, width, height]
    print(img.shape)
    print('===> Loading weights : ' + model_path)
    weight_dict = torch.load(model_path)  # 加载最后训练出的权重
    net.load_state_dict(weight_dict['state'])

    # 获取预测结果
    classes = [
        "0",
        "1",
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8",
        "9",
    ]

    
    net.eval()
    with torch.no_grad():
        predy = net(img)
        # 得到预测类别中最高的那一类，再把最高的这一类对应classes中的哪一类标签

        # predicted, actual = classes[torch.argmax(predy[0])], classes[labely]
        # 最终输出的预测值与真实值
        # print(f'predicted: "{predicted}", actual:"{actual}"')
        np.set_printoptions(suppress = True) #非科学计数法
        # predy[0] = F.softmax(predy[0],dim=1)
        for x, y in predy[1].items():
            if x == 'softmax_output':
                print(f'predicted: "{y.numpy()}", actual:"{labely}"')


In [94]:
net = LeNet()
model_path = './checkpoint/model_14_2.pth' #  ***为指定加载的权重文件名称
file_path = './test_images/pic_6_23534.png'
# file_path = './test_images/padpic_9_13429.png'
labely = 6
# predict()
predictOneTensor(model_path, file_path, labely)

(28, 28)
torch.Size([1, 1, 28, 28])
===> Loading weights : ./checkpoint/model_14_2.pth
predicted: "[[0.00040186 0.         0.00000441 0.         0.00001509 0.00000096
  0.99957746 0.         0.00000016 0.        ]]", actual:"6"


# 读取权重

In [7]:
import torch
import torchvision
import numpy as np
import os
from matplotlib import pyplot as plt
import pandas as pd

In [42]:
#输出每层网络参数信息
def variaes_show():
    net = LeNet()
    data_input = Variable(torch.randn(16,1,28,28))
    print(data_input.size())
    net(data_input)
    print(summary(net,(1,28,28)))

variaes_show()

# net = LeNet()
# summary(net,(1,28,28),batch_size=16,device="cpu")

torch.Size([16, 1, 28, 28])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              ReLU-2            [-1, 6, 28, 28]               0
         MaxPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              ReLU-5           [-1, 16, 10, 10]               0
         MaxPool2d-6             [-1, 16, 5, 5]               0
            Linear-7                  [-1, 120]          48,120
              ReLU-8                  [-1, 120]               0
            Linear-9                   [-1, 84]          10,164
             ReLU-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
          Softmax-12                   [-1, 10]               0
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
-----

In [15]:
def load_weight(model_path):
        
        print('===> Loading weights : ' + model_path)
        weight_dict = torch.load(model_path)  # 加载最后训练出的权重

        print('type: ' + str(type(weight_dict)))
        print('len: ' + str(len(weight_dict)))

        for k in weight_dict.keys():
                print('key: '+ k)

        # print(weight_dict['state'])
        # print(weight_dict['epoch'])
        print("================================")

        for key,value in weight_dict['state'].items():
                value_np = value.numpy()
                if not os.path.isdir('csv'):
                        os.mkdir('./csv')
                # np.savetxt("./csv/%s.csv" %(key), value_np,  delimiter=",")
                # pd.DataFrame(value_np).to_csv("./csv/%s.csv" %(key))
                # print(key, value.size())
                print(key)
                print('shape: '+ str(value_np.shape))
                if (value_np.ndim == 4):
                        (n_dim, _, _, _) = value_np.shape
                        value_2d = value_np.reshape(n_dim,-1)
                        np.savetxt("./csv/%s.csv" %(key), value_2d,  delimiter=",")
                        print("csv: ", value_2d.shape,'\n')
                elif (value_np.ndim == 3):
                        ndim, _, _ = value_np.shape
                        value_2d = value_np.reshape(n_dim,-1)
                        np.savetxt("./csv/%s.csv" %(key), value_2d,  delimiter=",")
                        print("csv: ", value_2d.shape,'\n')
                # elif (value_np.ndim == 2):
                #         ndim, _ = value_np.shape
                #         value_2d = value_np.reshape(n_dim,-1)
                #         print(value_2d.shape) 
                else :  
                        value_2d = value_np
                        np.savetxt("./csv/%s.csv" %(key), value_2d,  delimiter=",")
                        print("csv: ", value_2d.shape,'\n')             


        print("================================")
        print(type(weight_dict['state']))
        

state_path = './checkpoint/model_14.pth'
load_weight(state_path)

===> Loading weights : ./checkpoint/model_14.pth
type: <class 'dict'>
len: 2
key: state
key: epoch
block_1.0.weight
shape: (6, 1, 5, 5)
csv:  (6, 25) 

block_1.0.bias
shape: (6,)
csv:  (6,) 

block_1.3.weight
shape: (16, 6, 5, 5)
csv:  (16, 150) 

block_1.3.bias
shape: (16,)
csv:  (16,) 

block_2.0.weight
shape: (120, 400)
csv:  (120, 400) 

block_2.0.bias
shape: (120,)
csv:  (120,) 

block_2.2.weight
shape: (84, 120)
csv:  (84, 120) 

block_2.2.bias
shape: (84,)
csv:  (84,) 

block_2.4.weight
shape: (10, 84)
csv:  (10, 84) 

block_2.4.bias
shape: (10,)
csv:  (10,) 

<class 'collections.OrderedDict'>
